<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Transformations using UDF
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
UDF examples and exercises<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">
<b>
Spark has variety of built-in functions to aid in transformations. These are highly optimized to take advantage of the core engine optimizations. However, often there might be specific cases where the built-in functions do not provide the required transformation. This is where <i>User Defined Function (UDF)</i> can be used to define the specific logic that is required for the transformation.<br>
</b>
</span>
<span style="font-family:verdana; font-size:1.0em;"><br>
    <b>Following is included in this notebook</b>:
    <ul>
        <li>Createing a basic UDF function</li>
        <li>Registering the function with Spark</li>
        <li>Using the registered UDF</li>
        <li>Using the registered UDF with Spark SQL</li>
        <li>Measure performance for both built-in function and the UDF</li>
        <li>Exercise 1: Create UDF to convert IP Address from String to Numbers</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>IF RUNNING IN DATABRICK environment, set the next cell to True</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Transformation UDF").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Create a Basic UDF<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Basic function to split on 'e'
</span>

In [ ]:
def splitOnE(x):
    return x.split('e')

In [ ]:
splitOnE('this is a sample string')

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Registering Function with UDF<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
    <b>Define following for registering the function:</b><br>
    <ul>
        <li>Storing the returned UDF <i>splitStrUDF</i></li>
        <li>Name to register with SQL <i>splitStrSqlUDF</i></li>
        <li>Function name to register <i>splitOnE</i></li>
        <li>Return type from the function <i>StringType</i></li>
    </ul>
</span>

In [ ]:
from pyspark.sql.types import StringType

In [ ]:
splitStrUDF = spark.udf.register("splitStrSqlUDF", splitOnE, StringType())

In [ ]:
print(type(splitStrUDF))

## Create a dataframe with 10M rows, use SHA-1 to create a hash for each number

In [ ]:
from pyspark.sql.functions import sha1, rand
df = (spark.range(1, 10000 * 10 * 10 * 10)
  .withColumn("random_value", rand(seed=10).cast("string"))
  .withColumn("hash", sha1("random_value"))
  .drop("random_value")
)

In [ ]:
showDataFrame(df)

## Apply the UDF created above to the "hash" column in the dataframe

In [ ]:
augmentedDF = df.select("*", splitStrUDF("hash").alias("augmented"))

In [ ]:
showDataFrame(augmentedDF)

## SQL API and DataFrame<br>
<span style="font-family:verdana; font-size:1.0em;">
    Use the registered UDF name <b><i>splitStrSqlUDF</i></b> with the SQL API. We can then write SQL to get the same results as we got applying the UDF to the dataframe
</span>

In [ ]:
df.createOrReplaceTempView("dfTable")

In [ ]:
sqlDF = spark.sql("SELECT id, hash, splitStrSqlUDF(hash) as augmented FROM dfTable")

In [ ]:
showDataFrame(sqlDF)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
UDF Performance<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Compare the performance of a user defined function (UDF) v/s the built-in function<br>
    <ul>
        <li>Will use lambda to increment a value of each item in a column (will register this UDF)</li>
        <li>Use the col to select the column and add one to each row</li>
    </ul>
</span>

## Create a large dataframe of random values

In [ ]:
from pyspark.sql.functions import col, rand

df = (spark.range(0, 100 * 1000 * 1000)
  .withColumn("id", (col("id") / 1000).cast("integer"))
  .withColumn("random_float", rand())
)

In [ ]:
df.count()

In [ ]:
showDataFrame(df)

## Register UDF

In [ ]:
# using lambda to increment value
from pyspark.sql.types import FloatType
  
plusOneUDF = spark.udf.register("plusOneUDF", lambda x: x + 1, FloatType())

## Compare the two approaches - UDF v/s Built-in

In [ ]:
%timeit df.withColumn("incremented_float", plusOneUDF("random_float")).count()

In [ ]:
%timeit df.withColumn("incremented_float", col("random_float") + 1).count()

<span style="font-family:verdana; font-size:1.2em;">
Which one is faster? and by how much? <br>
</span>

<font color='tomato'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Exercise 1: Convert IP Address from String to Decimal<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
IP Addresses are quite often part of the data especially when dealing with log files. Also the are in string format <i>10.10.10.10</i>. It is efficient to store it in numerical format which is more efficient.<br>
    
IP Address: 10.10.10.10<br>
Numerical: 168430090<br>
    

If the input string is `1.2.3.4`, think of it like `A.B.C.D` where A is 1, B is 2, etc. Solve this with the following steps:

(A x 256^3) + (B x 256^2) + (C x 256) + D <br>
(1 x 256^3) + (2 x 256^2) + (3 x 256) + 4 <br>
116777216 + 131072 + 768 + 4 <br>
16909060

Implement the following:
    <ol>
        <li>Create a function to convert a string ip address return a long</li>
        <li>Register the function with UDF</li>
        <li>Apply the UDF to the dataframe provided and write the converted IP Address to parsedIP column</li>
        <li>Display the newly created dataframe after applying the UDF</li>
    </ol>
</span>

## Expect Results

123.123.123.123 --> 2071690107<br>
1.2.3.4 --> 16909060<br> 
127.0.0.1 --> 2130706433<br>
192.168.3.230 --> 3232236518

In [ ]:
df = spark.createDataFrame([["123.123.123.123"], ["1.2.3.4"], ["127.0.0.1"], ["192.168.3.230"]], ['ip'])

In [ ]:
showDataFrame(df)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Resources<br>

</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Take a look at the <a href="https://docs.databricks.com/spark/latest/spark-sql/udf-python.html" target="_blank">Databricks documentation for more details</a>
</span>